<a href="https://colab.research.google.com/github/kenwkliu/ideas/blob/master/colab/HKMA-OpenAPIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Visualize the data from HKMA APIs
# https://apidocs.hkma.gov.hk/

import pandas as pd

# interative map
import folium

# bypass SSL cert
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None)):
  ssl._create_default_https_context = ssl._create_unverified_context

# Google colab interactive table
%load_ext google.colab.data_table   

In [ ]:
# Read the Coin Cart location and schedule
url = 'https://api.hkma.gov.hk/public/coin-cart-schedule?lang=en&sortby=start_date&sortorder=asc'

df = pd.read_json(url)
df

In [ ]:
# Read the record result into a DataFrame
json = df.loc['records', 'result']
cartDf = pd.DataFrame(json)
cartDf

In [ ]:
# Create an interactive folium map and zoom to HK
HK_LAT, HK_LONG = 22.3, 114.2
fmap = folium.Map(location=[HK_LAT, HK_LONG], zoom_start=11)

# From each row of the Cart dataframe, mark the map with location, address and date by a tooltip 
for row in cartDf.itertuples():
  tip = row.address + ": " + row.start_date + " to " + row.end_date
  folium.Marker([row.latitude, row.longitude], tooltip=tip).add_to(fmap)

fmap

In [ ]:
# save the map
fmap.save('coinCart.html')

In [ ]:
# Get the daily exchange rate
url = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/er-ir/er-eeri-daily'

# Read into a dataframe 
df = pd.read_json(url)
ccyDf = pd.DataFrame(df.loc['records', 'result'])
ccyDf.set_index('end_of_day', inplace=True)

# Display the major ccy
ccyDf[['usd', 'gbp', 'cad', 'aud', 'cny', 'jpy', 'eur']]

In [ ]:
# sort the date in asc order and plot the graph
ccyDf[['aud']].sort_index().plot(figsize = (12, 8))

In [ ]:
''' 
- Other HKMA examples: https://apidocs.hkma.gov.hk/apidata/
- HK gov open data: https://data.gov.hk/en/
  Example 11-a-side Soccer Pitches (Artificial Turf Pitch): https://www.lcsd.gov.hk/datagovhk/facility/facility-sp11atp.json

'''
